In [ ]:
import json
import openai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [54]:
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def saudacao_por_periodo(hora):
    if 5 <= hora < 12:
        return json.dumps({"saudacao":"Bom dia, Fuleiro!"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao":"Boa tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao": "Boa noite"})
    else:
        return json.dumps({"saudacao": "Boa madrugada"})

In [56]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_periodo",
            "description": "Retorna uma saudação baseada na hora do dia",
            "parameters":{
                "type": "object",
                "properties":{
                    "hora":{
                        "type":"integer",
                        "description":"A gira di dua en formato de 24h"},
                    },
                    "required":["hora"]
                
            }

        }
    }
]

In [57]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo
}

In [58]:
mensagens = [{"role": "user", "content":"Qual saudacao o modelo me da se for 10h?"}]

In [59]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=mensagens,
    tools=tools,
    tool_choice="auto"
)


In [60]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_rYmK8MyF8NSFasyRSXJWXuNu', function=Function(arguments='{"hora":10}', name='saudacao_por_periodo'), type='function')])

In [61]:
tool_calls = mensagem_resp.tool_calls
tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_rYmK8MyF8NSFasyRSXJWXuNu', function=Function(arguments='{"hora":10}', name='saudacao_por_periodo'), type='function')]

In [62]:
if tool_calls:
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )
        mensagens.append(
            {
            "tool_call_id": tool_call.id,
            "role":"tool",
            "name": function_name,
            "content": function_response
            }
        )
    segunda_resposta = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = mensagens
    )

In [63]:
resposta = segunda_resposta.choices[0].message
resposta


ChatCompletionMessage(content='Se for 10h, a saudação que eu te daria é: "Bom dia, Fuleiro!"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)